In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Input,Activation,BatchNormalization,MaxPooling2D,AveragePooling2D,ZeroPadding2D,Add,Dropout
from tensorflow.keras import Input,Model
from tensorflow.keras.initializers import GlorotUniform as glorot_uniform
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def identity_block(X, f, filters, stage, block,l):
    """
    Implementation of the identity block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0), kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [6]:
def convolutional_block(X, f, filters, stage, block, s = 2,l = 0.01):
    """
    Implementation of the convolutional block
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    

    # Second component of main path 
    X = Conv2D(F2, (f,f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer=glorot_uniform(seed =0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu') (X)

    # Third component of main path 
    X = Conv2D(F3, (1,1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer=glorot_uniform(seed =0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(F3, (1,1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer=glorot_uniform(seed =0),kernel_regularizer = tf.keras.regularizers.l2( l=l))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation 
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [7]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes
    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1,l = 0)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b',l=0)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c',l=0)


    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2,l = 0)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b',l = 0)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c',l = 0)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d',l = 0)

    # Stage 4 
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2,l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e',l = 0)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f',l = 0)

    # Stage 5 
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2,l = 0)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b',l = 0)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c',l = 0)

    # AVGPOOL . Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D()(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0),kernel_regularizer = tf.keras.regularizers.l2( l=0.01))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    

    return model

In [8]:
model = ResNet50(input_shape = (20,20,3),classes = 2)

In [9]:
model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

In [10]:
class DataScience:
    @staticmethod
    def data_frame(path):
        df = pd.read_csv(path)
        return df
    @staticmethod
    def data_set(path,cl_drop,width,height,depth,val_size,shuffle):
        df = DataScience.data_frame(path)
        X = df.drop(cl_drop,axis = 1).to_numpy()
        y = df[cl_drop].to_numpy()
        X = X.reshape(-1,width,height,depth)
        y = y.reshape(-1,1)
        
        X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = val_size,shuffle = shuffle)
        
        return (X_train,X_val,y_train,y_val)
    @staticmethod
    def set_data_frame(id,y_pred):
        id = id.reshape(-1,1)
        final = np.concatenate([id,y_pred],axis = 1)
        df_final = pd.DataFrame(final,columns = ['id','labels'])
        return df_final
    def set_y_predict(y_pred):
        y_pred_final = []
        for i in range(0,y_pred.shape[0]):
          if(y_pred[i][0]>y_pred[i][1]):
            y_pred_final.append(0)
          else:
            y_pred_final.append(1)
        y_pred_final = np.array(y_pred_final)
        y_pred_final = y_pred_final.reshape(-1,1)
        return y_pred_final
        

In [11]:
def f1_score(y,y_pred):
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  n = y.shape[0]
  for i in range(0,n):
    if(y_pred[i] == 1):
      if(y_pred[i] == y[i]):
        tp+=1
      else:
        fp+=1
    else:
      if(y_pred[i] == y[i]):
        tn+=1
      else:
        fn+=1
  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = (2*precision*recall)/(precision+recall)
  return f1

In [12]:
(X_train,X_val,y_train,y_val) = DataScience.data_set('/content/drive/MyDrive/Dataset/train.csv', 
                                                     'labels',20,20,3,0.4,True)

In [13]:
NUM_EPOCHS = 100
model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_data = (X_val,y_val))

Epoch 1/100
99/99 [==============================] - 56s 67ms/step - loss: 1.2768 - accuracy: 0.6657 - val_loss: 0.6142 - val_accuracy: 0.7410
Epoch 2/100
99/99 [==============================] - 5s 47ms/step - loss: 0.5295 - accuracy: 0.7616 - val_loss: 0.6169 - val_accuracy: 0.7410
Epoch 3/100
99/99 [==============================] - 5s 48ms/step - loss: 0.4116 - accuracy: 0.8184 - val_loss: 0.6333 - val_accuracy: 0.7424
Epoch 4/100
99/99 [==============================] - 4s 43ms/step - loss: 0.3080 - accuracy: 0.8759 - val_loss: 0.6448 - val_accuracy: 0.7890
Epoch 5/100
99/99 [==============================] - 5s 55ms/step - loss: 0.2182 - accuracy: 0.9222 - val_loss: 0.5891 - val_accuracy: 0.7990
Epoch 6/100
99/99 [==============================] - 4s 43ms/step - loss: 0.1613 - accuracy: 0.9483 - val_loss: 0.6536 - val_accuracy: 0.7619
Epoch 7/100
99/99 [==============================] - 4s 43ms/step - loss: 0.1371 - accuracy: 0.9575 - val_loss: 0.7363 - val_accuracy: 0.7757
Epoch

In [14]:
y_val_predict = model.predict(X_val)
y_val_predict = DataScience.set_y_predict(y_val_predict)

66/66 [==============================] - 1s 9ms/step


In [15]:
f1_score(y_val,y_val_predict)

0.5754527162977867

In [16]:
df_test = pd.read_csv('/content/drive/MyDrive/Dataset/test.csv')
X_test = df_test.drop('id',axis = 1).to_numpy()
id =  df_test['id'].to_numpy()
X_test = X_test.reshape(-1,20,20,3)

In [17]:
y_test_pred = model.predict(X_test)
y_test_pred = DataScience.set_y_predict(y_test_pred)
(DataScience.set_data_frame(id,y_test_pred)).to_csv('submission6.csv')

71/71 [==============================] - 1s 15ms/step


In [19]:
model2 = ResNet50(input_shape = (20,20,3),classes = 2)

In [20]:
model2.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy',metrics = ['accuracy'])

In [21]:
model2.fit(X_train,y_train,validation_data = (X_val,y_val),epochs = 150)

Epoch 1/150
99/99 [==============================] - 58s 103ms/step - loss: 1.0047 - accuracy: 0.6848 - val_loss: 0.6066 - val_accuracy: 0.7410
Epoch 2/150
99/99 [==============================] - 8s 78ms/step - loss: 0.4726 - accuracy: 0.7756 - val_loss: 0.5822 - val_accuracy: 0.7410
Epoch 3/150
99/99 [==============================] - 5s 48ms/step - loss: 0.3776 - accuracy: 0.8457 - val_loss: 0.5479 - val_accuracy: 0.7481
Epoch 4/150
99/99 [==============================] - 4s 44ms/step - loss: 0.2872 - accuracy: 0.8984 - val_loss: 0.5208 - val_accuracy: 0.7924
Epoch 5/150
99/99 [==============================] - 6s 58ms/step - loss: 0.1964 - accuracy: 0.9302 - val_loss: 0.6261 - val_accuracy: 0.7833
Epoch 6/150
99/99 [==============================] - 4s 44ms/step - loss: 0.2015 - accuracy: 0.9321 - val_loss: 0.6543 - val_accuracy: 0.7481
Epoch 7/150
99/99 [==============================] - 6s 61ms/step - loss: 0.4021 - accuracy: 0.8644 - val_loss: 1.5487 - val_accuracy: 0.8019
Epoc

In [25]:
y_val_predict_2 = model.predict(X_val)
y_val_predict_2 = DataScience.set_y_predict(y_val_predict_2)

66/66 [==============================] - 1s 9ms/step


In [26]:
f1_score(y_val,y_val_predict_2)

0.5754527162977867

In [24]:
y_test_pred_2 = model.predict(X_test)
y_test_pred_2 = DataScience.set_y_predict(y_test_pred_2)
(DataScience.set_data_frame(id,y_test_pred)).to_csv('submission7.csv')

71/71 [==============================] - 1s 11ms/step
